In [ ]:
pip install blpapi

In [ ]:
import blpapi
# Import the Bloomberg API library, which provides the classes and functions needed to interact with Bloomberg services.

# Create a session
sessionOptions = blpapi.SessionOptions()
# Initialize a SessionOptions object to specify connection options for the Bloomberg API session.

sessionOptions.setServerHost('localhost')
# Set the server host to 'localhost', indicating that the Bloomberg Terminal is running locally on the same machine.

sessionOptions.setServerPort(8194)
# Set the server port to 8194, the default port for Bloomberg API connections.

session = blpapi.Session(sessionOptions)
# Create a session object with the specified session options to manage the connection to Bloomberg services.

if not session.start():
    print("Failed to start session.")
    exit()
# Start the session. If the session fails to start, print an error message and exit the script.

if not session.openService("//blp/mktdata"):
    print("Failed to open //blp/mktdata")
    exit()
# Open the Bloomberg Market Data service. If it fails to open, print an error message and exit the script.

service = session.getService("//blp/mktdata")
# Retrieve the market data service object from the session.

request = service.createRequest("MarketDataRequest")
# Create a market data request object to specify the data needed from Bloomberg.

# Add securities and fields to the request
request.append("securities", "IBM US Equity")
# Append the security identifier for IBM US Equity to the request.

request.append("fields", "PX_LAST")
# Append the field 'PX_LAST' (last price) to the request, specifying the type of market data needed.

# Send the request
session.sendRequest(request)
# Send the market data request to Bloomberg.

# Process received events
while True:
    event = session.nextEvent()
    # Wait for and retrieve the next event from the session.

    for msg in event:
        print(msg)
    # Iterate through the messages in the event and print each message.

    if event.eventType() == blpapi.Event.RESPONSE:
        break
    # If the event type is RESPONSE, it indicates the end of the response messages, and we can break the loop.

# Stop the session
session.stop()
# Stop the session to clean up and release resources.

In [ ]:
import blpapi
from blpapi import Service
from datetime import datetime
import pandas as pd

# Initialize Bloomberg session
session_options = blpapi.SessionOptions()
session_options.setServerHost('localhost')  # Set the server host for the Bloomberg session
session_options.setServerPort(8194)  # Set the server port for the Bloomberg session
session = blpapi.Session(session_options)  # Create a new Bloomberg session with the specified options

if not session.start():
    print("Failed to start session.")  # Print an error message if the session fails to start
    exit()

if not session.openService("//blp/refdata"):
    print("Failed to open //blp/refdata")  # Print an error message if the reference data service fails to open
    exit()

refDataService = session.getService("//blp/refdata")  # Get the reference data service
request = refDataService.createRequest("HistoricalDataRequest")  # Create a request for historical data

# Define request parameters
request.getElement("securities").appendValue("AAPL US Equity")  # Specify the security for which to retrieve data (Apple Inc.)
request.getElement("fields").appendValue("PX_LAST")  # Specify the field to retrieve (last price)
request.set("startDate", "20230101")  # Set the start date for the data retrieval
request.set("endDate", "20231231")  # Set the end date for the data retrieval
request.set("periodicitySelection", "DAILY")  # Set the periodicity of the data (daily)

# Send the request
session.sendRequest(request)  # Send the request to Bloomberg

# Process the response
end_reached = False
while not end_reached:
    event = session.nextEvent(500)  # Wait for the next event (up to 500 milliseconds)
    for msg in event:
        if msg.asElement().name() == "HistoricalDataResponse":
            security_data = msg.getElement("securityData")  # Get the security data from the response
            field_data = security_data.getElement("fieldData")  # Get the field data from the security data
            data = []
            for fd in field_data.values():
                date = fd.getElementAsDatetime("date")  # Extract the date from the field data
                price = fd.getElementAsFloat("PX_LAST")  # Extract the price from the field data
                data.append([date, price])  # Append the date and price to the data list
            df = pd.DataFrame(data, columns=["Date", "Price"])  # Create a DataFrame from the data
            df.set_index("Date", inplace=True)  # Set the date as the index of the DataFrame
            print(df)  # Print the DataFrame
            end_reached = True  # Indicate that the end of the response has been reached
session.stop()  # Stop the Bloomberg session

In [ ]:
import blpapi
from blpapi import Service
import pandas as pd

# Initialize Bloomberg session
session_options = blpapi.SessionOptions()
session_options.setServerHost('localhost')  # Set the server host for the Bloomberg session
session_options.setServerPort(8194)  # Set the server port for the Bloomberg session
session = blpapi.Session(session_options)  # Create a new Bloomberg session with the specified options

if not session.start():
    print("Failed to start session.")  # Print an error message if the session fails to start
    exit()

if not session.openService("//blp/refdata"):
    print("Failed to open //blp/refdata")  # Print an error message if the reference data service fails to open
    exit()

refDataService = session.getService("//blp/refdata")  # Get the reference data service
request = refDataService.createRequest("ReferenceDataRequest")  # Create a request for reference data

# Define request parameters
request.getElement("securities").appendValue("AAPL US Equity")  # Specify the security for which to retrieve data (Apple Inc.)
request.getElement("fields").appendValue("OPT_CHAIN")  # Specify the field to retrieve (option chain)

overrides = request.getElement("overrides")  # Get the overrides element of the request
override = overrides.appendElement()  # Append a new override element
override.setElement("fieldId", "CHAIN_DATE")  # Set the field ID for the override (chain date)
override.setElement("value", "20240119")  # Set the value for the override (specific date)

# Send the request
session.sendRequest(request)  # Send the request to Bloomberg

# Process the response
end_reached = False
while not end_reached:
    event = session.nextEvent(500)  # Wait for the next event (up to 500 milliseconds)
    for msg in event:
        if msg.asElement().name() == "ReferenceDataResponse":
            securities = msg.getElement("securityData").values()  # Get the security data from the response
            for security in securities:
                field_data = security.getElement("fieldData")  # Get the field data from the security data
                option_chains = field_data.getElement("OPT_CHAIN")  # Get the option chain data from the field data
                data = []
                for option in option_chains.values():
                    ticker = option.getElementAsString("Security Description")  # Extract the security description
                    strike = option.getElementAsFloat("Strike Price")  # Extract the strike price
                    option_type = option.getElementAsString("Option Type")  # Extract the option type
                    expiration = option.getElementAsDatetime("Expiration Date")  # Extract the expiration date
                    data.append([ticker, strike, option_type, expiration])  # Append the extracted data to the data list
                df = pd.DataFrame(data, columns=["Ticker", "Strike", "Option Type", "Expiration"])  # Create a DataFrame from the data
                print(df)  # Print the DataFrame
                end_reached = True  # Indicate that the end of the response has been reached
session.stop()  # Stop the Bloomberg session

In [ ]:
import blpapi
from blpapi import Service
import pandas as pd

# Initialize Bloomberg session
session_options = blpapi.SessionOptions()
session_options.setServerHost('localhost')
session_options.setServerPort(8194)
session = blpapi.Session(session_options)

if not session.start():
    print("Failed to start session.")
    exit()

if not session.openService("//blp/refdata"):
    print("Failed to open //blp/refdata")
    exit()

refDataService = session.getService("//blp/refdata")
request = refDataService.createRequest("ReferenceDataRequest")

# Define request parameters
request.getElement("securities").appendValue("AAPL US Equity")  # Example: Apple Inc.

# Add a wide range of fields
fields = [
    "PX_LAST", "PX_OPEN", "PX_HIGH", "PX_LOW", "PX_VOLUME", "PX_CLOSE",
    "RETURN_COM_EQY", "VWAP", "PE_RATIO", "DIVIDEND_YIELD", "52_WEEK_HIGH",
    "52_WEEK_LOW", "MARKET_CAP", "EPS", "BETA", "DIVIDEND_AMOUNT",
    "DIVIDEND_FREQUENCY", "NAME", "TICKER", "COUNTRY", "INDUSTRY_SECTOR",
    "INDUSTRY_GROUP", "MARKET_SECTOR_DES", "CRNCY", "SECURITY_TYP",
    "ISSUE_DT", "MATURITY", "DURATION", "COUNTRY_ISO"
]

for field in fields:
    request.getElement("fields").appendValue(field)

# Send the request
session.sendRequest(request)

# Process the response
end_reached = False
while not end_reached:
    event = session.nextEvent(500)
    for msg in event:
        if msg.asElement().name() == "ReferenceDataResponse":
            securities = msg.getElement("securityData").values()
            for security in securities:
                field_data = security.getElement("fieldData")
                data = {}
                for field in fields:
                    if field_data.hasElement(field):
                        data[field] = field_data.getElementAsString(field)
                    else:
                        data[field] = None
                df = pd.DataFrame([data])
                print(df)
                end_reached = True
session.stop()

In [ ]:
Equity Data Fields
PX_LAST: Last Price
PX_OPEN: Open Price
PX_HIGH: High Price
PX_LOW: Low Price
PX_VOLUME: Volume
PX_CLOSE: Close Price
RETURN_COM_EQY: Total Return
VWAP: Volume Weighted Average Price
PE_RATIO: Price/Earnings Ratio
DIVIDEND_YIELD: Dividend Yield
52_WEEK_HIGH: 52 Week High
52_WEEK_LOW: 52 Week Low
MARKET_CAP: Market Capitalization
EPS: Earnings Per Share
BETA: Beta
DIVIDEND_AMOUNT: Dividend Amount
DIVIDEND_FREQUENCY: Dividend Frequency

Bond Data Fields
YLD_YTM_MID: Yield to Maturity
YLD_CNV_MID: Yield to Convention
YLD_CUR_MID: Current Yield
DUR_ADJ_OAS: Option-Adjusted Spread Duration
OAS_SPREAD: Option-Adjusted Spread
Z_SPREAD: Zero-Volatility Spread
MATURITY: Maturity Date
COUPON: Coupon Rate
DURATION: Duration
ISSUE_PX: Issue Price
RATING_MOODY: Moody's Rating
RATING_SP: S&P Rating

Option Data Fields
OPT_CHAIN: Option Chain
OPT_UNDERLYING_TICKER: Underlying Ticker
OPT_STRIKE: Strike Price
OPT_EXPIRE_DT: Expiration Date
OPT_PUT_CALL: Put/Call Indicator
OPT_LAST_TRADE: Last Trade Price
OPT_OPEN_INT: Open Interest
OPT_IMP_VOL: Implied Volatility
OPT_DELTA: Delta
OPT_GAMMA: Gamma
OPT_VEGA: Vega
OPT_THETA: Theta

FX Data Fields
PX_LAST: Last Price
PX_HIGH: High Price
PX_LOW: Low Price
PX_CLOSE: Close Price
VOLUME: Trading Volume
SPOT: Spot Rate
FORWARD_POINTS: Forward Points
FX_SPOT_RATE: FX Spot Rate
FX_FORWARD_RATE: FX Forward Rate

Metadata Fields
NAME: Security Name
TICKER: Ticker Symbol
COUNTRY: Country of Issuance
INDUSTRY_SECTOR: Industry Sector
INDUSTRY_GROUP: Industry Group
MARKET_SECTOR_DES: Market Sector Description
CRNCY: Currency
SECURITY_TYP: Security Type
ISSUE_DT: Issue Date
MATURITY: Maturity Date
DURATION: Duration
COUNTRY_ISO: Country ISO Code
Detailed Example with Extensive

In [ ]:
import blpapi
from blpapi import Service
import pandas as pd

# Initialize Bloomberg session
session_options = blpapi.SessionOptions()
session_options.setServerHost('localhost')
session_options.setServerPort(8194)
session = blpapi.Session(session_options)

if not session.start():
    print("Failed to start session.")
    exit()

if not session.openService("//blp/refdata"):
    print("Failed to open //blp/refdata")
    exit()

refDataService = session.getService("//blp/refdata")
request = refDataService.createRequest("ReferenceDataRequest")

# Define request parameters
request.getElement("securities").appendValue("AAPL US Equity")  # Example: Apple Inc.

# Add GICS fields
fields = ["GICS_SECTOR_NAME", "GICS_INDUSTRY_GROUP_NAME", "GICS_INDUSTRY_NAME", "GICS_SUB_INDUSTRY_NAME"]

for field in fields:
    request.getElement("fields").appendValue(field)

# Send the request
session.sendRequest(request)

# Process the response
end_reached = False
while not end_reached:
    event = session.nextEvent(500)
    for msg in event:
        if msg.asElement().name() == "ReferenceDataResponse":
            securities = msg.getElement("securityData").values()
            for security in securities:
                field_data = security.getElement("fieldData")
                data = {}
                for field in fields:
                    if field_data.hasElement(field):
                        data[field] = field_data.getElementAsString(field)
                    else:
                        data[field] = None
                df = pd.DataFrame([data])
                print(df)
                end_reached = True
session.stop()

In [ ]:
import blpapi
from xbbg import blp
import pandas as pd

# Ensure that Bloomberg Terminal is running and you are connected
sessionOptions = blpapi.SessionOptions()
sessionOptions.setServerHost('localhost')
sessionOptions.setServerPort(8194)

session = blpapi.Session(sessionOptions)

if not session.start():
    print("Failed to start session.")
    exit()

if not session.openService("//blp/mktdata"):
    print("Failed to open //blp/mktdata")
    exit()

print("Connected to Bloomberg Terminal")

# Define your data directory, tickers, fields, and date range
DATA_DIR = './data/'

tickers = ['NVDA US Equity', 'AAPL US Equity']
fields = ['High', 'Low', 'Last_Price']
start_date = '2023-09-01'
end_date = '2023-09-20'

# Fetch historical tick data
hist_tick_data = blp.bdh(tickers=tickers, flds=fields, start_date=start_date, end_date=end_date)

# Define filename and save the data to a CSV file
filename = f'tick_data_{start_date}_{end_date}.csv'
hist_tick_data.to_csv(DATA_DIR + filename)

print(f"Data saved to {DATA_DIR + filename}")